In [ ]:
# CELL
import os, cv2, numpy as np, matplotlib.pyplot as plt, scipy.io as sio
plt.rcParams["figure.figsize"] = (7,6)

In [ ]:
# ====== EDIT THESE ======
INPUT_IMAGE = r"C:\Users\mahal\OneDrive\Desktop\DL\archive\ShanghaiTech\part_A\train_data\images\IMG_1.jpg"
GT_FILE = r"C:\Users\mahal\OneDrive\Desktop\DL\archive\ShanghaiTech\part_A\train_data\ground_truth\GT_IMG_1.mat"
DOWNSAMPLE_FACTOR = 0.5
TARGET_SIZE = (512,512)   # (W,H)
SIGMA = 15.0
# =======================

In [ ]:
def show(img, title=None, cmap=None):
    plt.figure(); 
    if img.ndim==3: plt.imshow(img); 
    else: plt.imshow(img, cmap=cmap if cmap else "viridis")
    if title: plt.title(title)
    plt.axis("off"); plt.show()

In [ ]:
# CELL 1: read and show original (RGB)
assert os.path.exists(INPUT_IMAGE), "Set INPUT_IMAGE to an existing file"
bgr = cv2.imread(INPUT_IMAGE, cv2.IMREAD_COLOR)
if bgr is None: raise RuntimeError("Failed to read: "+INPUT_IMAGE)
rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
show(rgb, f"Original (RGB) {rgb.shape}")

In [ ]:
# CELL 2: BGR->RGB conversion (explicit)
rgb_conv = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
show(rgb_conv, "After BGR->RGB")

In [ ]:
# CELL 3: downsample
h0,w0 = rgb_conv.shape[:2]
dw = max(1, int(w0*DOWNSAMPLE_FACTOR)); dh = max(1, int(h0*DOWNSAMPLE_FACTOR))
img_down = cv2.resize(rgb_conv, (dw, dh), interpolation=cv2.INTER_AREA)
show(img_down, f"Downsampled (factor={DOWNSAMPLE_FACTOR}) {img_down.shape}")

In [ ]:
# CELL 4: resize to TARGET_SIZE
TW, TH = TARGET_SIZE
img_resized = cv2.resize(img_down, (TW, TH), interpolation=cv2.INTER_AREA)
show(img_resized, f"Resized to {TARGET_SIZE}")

In [ ]:
# CELL 5: robust GT loader and overlay annotations on resized image
def load_points_from_mat(path):
    data = sio.loadmat(path)
    # Try ShanghaiTech nesting: image_info[0][0][0][0][0]
    try:
        if 'image_info' in data:
            cand = data['image_info'][0][0][0][0][0]
            pts = np.array(cand).squeeze()
            return pts.reshape(-1,2).astype(np.float32)
    except Exception:
        pass
    # Try other typical nests inside image_info
    try:
        if 'image_info' in data:
            info = data['image_info'][0][0]
            if hasattr(info, 'dtype') and info.dtype.names:
                for key in ('location','annPoints','points','gt','GT'):
                    if key in info.dtype.names:
                        val = info[key]
                        arr = np.array(val).squeeze()
                        return arr.reshape(-1,2).astype(np.float32)
    except Exception:
        pass
    # Top-level Nx2 fallback
    for k,v in data.items():
        if isinstance(v, np.ndarray) and v.ndim==2 and v.shape[1]==2:
            return v.astype(np.float32)
    # last resort: flattenable
    for v in data.values():
        if isinstance(v, np.ndarray):
            flat = v.flatten()
            if flat.size>=2 and flat.size%2==0:
                try:
                    return flat.reshape(-1,2).astype(np.float32)
                except Exception:
                    pass
    raise ValueError("No Nx2 points found in MAT. Keys: "+",".join(data.keys()))

In [ ]:
def load_points(path):
    if path is None: return None
    if not os.path.exists(path): 
        print("GT not found:", path); return None
    ext = os.path.splitext(path)[1].lower()
    if ext == '.mat': return load_points_from_mat(path)
    if ext == '.npy': return np.load(path)
    if ext in ('.txt','.csv'):
        a = np.loadtxt(path, delimiter=None)
        if a.ndim==1: a = a.reshape(-1,2)
        return a.astype(np.float32)
    return None

In [ ]:
pts = load_points(GT_FILE)
if pts is None:
    print("No annotations loaded; set GT_FILE correctly.")
else:
    # scale points from original image -> resized image
    orig_h, orig_w = rgb.shape[:2]
    sx = TW / orig_w; sy = TH / orig_h
    pts_resized = pts.copy().astype(np.float32)
    pts_resized[:,0] = pts_resized[:,0] * sx
    pts_resized[:,1] = pts_resized[:,1] * sy
    # overlay
    overlay = img_resized.copy()
    for (x,y) in pts_resized:
        cx, cy = int(round(x)), int(round(y))
        if 0<=cx<TW and 0<=cy<TH:
            cv2.circle(overlay, (cx,cy), 3, (255,0,0), -1)  # red dots (RGB)
    show(overlay, f"Resized with head annotations (count={len(pts_resized)})")

In [ ]:
# CELL 6: density map for resized image (jet colormap, blue background)
if 'pts_resized' in globals() and pts_resized is not None:
    H,W = img_resized.shape[:2]
    density = np.zeros((H,W), dtype=np.float32)
    for (x,y) in pts_resized:
        ix, iy = int(round(x)), int(round(y))
        if 0<=ix<W and 0<=iy<H:
            density[iy,ix] += 1.0
    k = max(3, int(6*SIGMA) | 1)
    density = cv2.GaussianBlur(density, (k,k), sigmaX=SIGMA, sigmaY=SIGMA, borderType=cv2.BORDER_REPLICATE)
    vmax = np.percentile(density, 99)
    if vmax <= 0: vmax = density.max() if density.max()>0 else 1.0
    plt.figure(); plt.imshow(density, cmap='jet', vmin=0, vmax=vmax, interpolation='bilinear'); plt.title('Density map (sigma=%g)'%SIGMA); plt.axis('off'); plt.show()
    # overlay on resized
    norm = (density - density.min())/(vmax - density.min() + 1e-9)
    colored = plt.cm.jet(norm)[:,:,:3]
    alpha = 0.55
    overlay_img = (1-alpha)*(img_resized.astype(np.float32)/255.0) + alpha*colored
    overlay_img = np.clip(overlay_img, 0.0, 1.0)
    show(overlay_img, "Density overlay on resized image")
else:
    print("No pts_resized: run CELL 5 and ensure GT loaded correctly.")